In [3]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama import ChatOllama

In [4]:
# LLM load
llm = ChatOllama(
    model = "llama3.1",
    base_url = "http://localhost:11434")

In [5]:
## GENERATE

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

generate = prompt | llm
essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

The Little Prince, written by Antoine de Saint-Exupéry, is a timeless tale that has captivated readers for generations. Published in 1943, this novella has stood the test of time and remains a significant literary work for children today. Despite being written over seven decades ago, The Little Prince's themes, characters, and philosophical undertones remain remarkably relevant to modern childhood.

One reason why The Little Prince remains pertinent is its exploration of the human condition through the lens of a child's perspective. The story follows a young prince from a distant asteroid as he travels across the galaxy, encountering various strange creatures that have become disillusioned with their own unique gifts and talents. Through this narrative, de Saint-Exupéry masterfully conveys the universal struggles faced by children today: feeling isolated, misunderstood, or overlooked in a world dominated by adults. This resonance is particularly evident in the modern era, where issues 

In [6]:
## REFLEXION

reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content


Overall, your essay provides a clear and compelling argument for why The Little Prince remains relevant in modern childhood. Here are some specific strengths and areas for improvement:

**Strengths:**

1. **Clear thesis**: Your introduction effectively sets up the main argument that The Little Prince is still relevant today.
2. **Relevant examples**: You provide concrete examples from the book to support your claims, such as the universal struggles faced by children and the importance of empathy and kindness.
3. **Depth of analysis**: Your essay goes beyond mere summary and offers thoughtful insights into the themes and significance of The Little Prince.

**Areas for improvement:**

1. **Length**: While your essay is a good length, it could benefit from a bit more expansion to make each point more fully developed. Aim for 500-700 words.
2. **Transitions**: Some paragraphs feel a bit disconnected; consider using transitional phrases or sentences to link ideas between paragraphs.
3. **De

In [7]:
##REPEAT    
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")


Thank you for the detailed critique! I'll take each point into consideration and revise my essay to address the areas of improvement. Here's a rewritten version:

The Little Prince, Antoine de Saint-Exupéry's 1943 novella, has become an unlikely companion for many children navigating the complexities of modern childhood. Yet, despite its publication over seven decades ago, this deceptively simple story remains an essential read for today's young people. In fact, The Little Prince is more than just a charming tale – it's a mirror held up to the human experience, reflecting our deepest desires, fears, and struggles.

One of the primary reasons why The Little Prince continues to resonate with modern childhood is its uncanny ability to capture the universal struggles faced by children. Like the little prince himself, young readers today often feel isolated, misunderstood, or overlooked in a world dominated by adults. De Saint-Exupéry masterfully conveys this sentiment through his protagoni

In [ ]:
##### USING LANGGRAPH

In [10]:
import asyncio
from langgraph.graph import END, MessageGraph
from typing import Sequence
from langchain_core.messages import HumanMessage
from typing import List



from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an AI assistant researcher tasked with researching on a variety of topics in a short summary of 5 paragraphs."
            " Generate the best research possible as per user request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)



generate = prompt | llm

reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a senior researcher"
            " Provide detailed recommendations, including requests for length, depth, style, etc."
            " to an asistant researcher to help improve this researches",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

reflect = reflection_prompt | llm


async def generation_node(state: Sequence[BaseMessage]):
    return await generate.ainvoke({"messages": state})


async def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}

    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    res = await reflect.ainvoke({"messages": translated})

    # this will be treated as a feedback for the generator
    return HumanMessage(content=res.content)


builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.set_entry_point("generate")


def should_continue(state: List[BaseMessage]):
    if len(state) > 2:
        # End after 2 iterations
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()


import nest_asyncio
nest_asyncio.apply()

async def stream_responses():
    async for event in graph.astream(
        [
            HumanMessage(
                content="Research on climate change"
            )
        ],
    ):
        print(event)
        print("---")


asyncio.run(stream_responses())   
 



{'generate': AIMessage(content='**Climate Change: A Global Phenomenon**\n\nClimate change refers to the long-term warming of the planet due to an increase in average global temperatures. This phenomenon is primarily caused by human activities that release greenhouse gases, such as carbon dioxide (CO2) and methane (CH4), into the atmosphere, leading to a trap of heat and a rise in global temperatures. The consequences of climate change are far-reaching, affecting not only the environment but also human societies and economies.\n\n**Causes of Climate Change**\n\nThe main causes of climate change can be attributed to human activities that contribute to greenhouse gas emissions. These include:\n\n*   **Burning fossil fuels**: The combustion of coal, oil, and natural gas for energy releases massive amounts of CO2 into the atmosphere.\n*   **Deforestation**: The clearing of forests for agriculture, urbanization, and other purposes leads to a reduction in carbon sinks, allowing more CO2 to ac